<a href="https://colab.research.google.com/github/leolani/cltl-face-all/blob/master/examples/colab/1.visual-features-extraction-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Clone the repo, install the package, download data, and extract them.

Don't run `pip install -r requirements.txt` from the original installation guide. They are made for python3.7, but colab still uses python3.6. We'll install the necessary packages here on the fly.

In [5]:
%cd /content
!git clone https://github.com/leolani/cltl-face-all
!cd cltl-face-all/ && pip install .
!pip install omegaconf==2.0.5
!pip install tqdm av

# Download the annotations and stuff
!wget https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/train_sent_emo.csv
!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datasets-jsons/dataset-small.json
!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datasets-jsons/dataset-medium.json
!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datasets-jsons/dataset-large.json
!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datasets-jsons/datasets.json

!gdown --id 16ck7plW9v9eSHGCs5wuB2AhhufPRt3Wi
!unzip smaller-dataset.zip
!rm smaller-dataset.zip
!ls smaller-dataset/  |  wc -l

/content
fatal: destination path 'cltl-face-all' already exists and is not an empty directory.
Processing /content/cltl-face-all
  Created wheel for cltl-face-all: filename=cltl_face_all-0.0.0-cp36-none-any.whl size=53234 sha256=697b9bae5c78e5bf3fa6a0f3453c29c77ff673a4332ab60d98085cfcf093eac6
  Stored in directory: /root/.cache/pip/wheels/8d/71/f3/2db0b17856a0b395aaa04c8562716e0350cfc712cdd4b9947e
Successfully built cltl-face-all
  Found existing installation: cltl-face-all 0.0.0
    Uninstalling cltl-face-all-0.0.0:
      Successfully uninstalled cltl-face-all-0.0.0
--2020-12-03 18:41:21--  https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/train_sent_emo.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1119290 (1.1M) [text/plain]
Saving to: 

# Assess the data

The number of videos should be the same

In [6]:
import json

with open('datasets.json', 'r') as stream:
    datasets = json.load(stream)

datasets = datasets['large']

# Let's check out a couple of videos to see if our labeling is correct.

In [14]:
from glob import glob
import random
import os
import json
import numpy as np
import av
import matplotlib.pyplot as plt
import cv2
import random
from IPython.display import HTML
from base64 import b64encode

VIDEOS_PATH = "smaller-dataset/"

diautt = random.choice([foo for foo, bar in datasets['train'].items()])
annot = datasets['train'][diautt]
print(annot)
vid_path = os.path.join(VIDEOS_PATH, diautt) +  '.mp4'

mp4 = open(vid_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=1280 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

{'SrNo': '2304', 'Utterance': "I'm gonna go talk to uh, a friend.", 'Speaker': 'Tom', 'Emotion': 'sadness', 'Sentiment': 'negative', 'Dialogue_ID': '226', 'Utterance_ID': '17', 'Season': '5', 'Episode': '7', 'StartTime': '00:19:41,805', 'EndTime': '00:19:44,473'}


# Instantiate the three classes we need.

In [15]:
from cltl_face_all.face_alignment import FaceDetection
from cltl_face_all.arcface import ArcFace
from cltl_face_all.agegender import AgeGender

fd = FaceDetection(device='cuda', face_detector='sfd')
ag = AgeGender(device='cpu')
af = ArcFace(device='cpu')

Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth


Downloading: "https://www.adrianbulat.com/downloads/python-fan/2DFAN4-11f355bf06.pth.tar" to /root/.cache/torch/hub/checkpoints/2DFAN4-11f355bf06.pth.tar



94773248/94765736 [==============================] - 1s 0us/step
[*] load ckpt from /usr/local/lib/python3.6/dist-packages/cltl_face_all/arcface/./pretrained_models/arc_res50/e_8_b_40000.ckpt


# Run over the images

In [ ]:
from tqdm.notebook import tqdm
import av
import numpy as np
import os
import shutil

!rm visual-features.zip
shutil.rmtree('visual-features', ignore_errors=True)
os.makedirs('visual-features', exist_ok=True)

for vidpath in tqdm(glob(os.path.join(VIDEOS_PATH, '*.mp4'))):
    print(vidpath)

    diautt = os.path.basename(vidpath)
    to_dump = {}

    container = av.open(vidpath)

    for idx, frame in enumerate(container.decode(video=0)):
        to_dump[idx] = []
        numpy_RGB = np.array(frame.to_image())
        batch = numpy_RGB[np.newaxis, ...]
        bboxes = fd.detect_faces(batch)
        # print(f"number of faces detected in the frame {idx} is {len(bboxes[0])}")

        if len(bboxes[0]) == 0:
            continue

        landmarks = fd.detect_landmarks(batch, bboxes)

        faces = fd.crop_and_align(batch, bboxes, landmarks)
        faces = np.concatenate(faces, axis=0)
        embeddings = af.predict(faces)
        ages, genders = ag.predict(faces)

        # print(len(bboxes[0]), len(landmarks[0]), len(ages), len(genders), len(embeddings), len(faces))
        for bb, lm, a, g, emb in zip(bboxes[0], landmarks[0], ages, genders, embeddings):
            x1, y1, x2, y2, prob = bb

            to_append = {'bbox': bb, 
                        'landmark': lm, 
                        'age': a,
                        'gender': g,
                        'embedding': emb}

            to_dump[idx].append(to_append)
        # print(len(to_dump), idx+1)

    np.save(os.path.join('visual-features', f"{diautt.replace('.mp4', '.npy')}" ), to_dump)

!zip -r visual-features.zip visual-features
!cp visual-features.zip drive/MyDrive/

rm: cannot remove 'visual-features.zip': No such file or directory


smaller-dataset/dia226_utt3.mp4


/usr/local/lib/python3.6/dist-packages/cltl_face_all/face_alignment/matlab_cp2tform.py:84: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r, _, _, _ = lstsq(X, U)


smaller-dataset/dia982_utt3.mp4
smaller-dataset/dia805_utt3.mp4
smaller-dataset/dia990_utt14.mp4
smaller-dataset/dia536_utt0.mp4
smaller-dataset/dia939_utt1.mp4
smaller-dataset/dia990_utt2.mp4
smaller-dataset/dia87_utt3.mp4
smaller-dataset/dia441_utt9.mp4
smaller-dataset/dia990_utt12.mp4
smaller-dataset/dia111_utt17.mp4
